## Setup configuration for our validation

In [19]:
HD_OUTPUT_FILENAME = 'hashdice_validation.txt'
PREVIOUS_TIMESTEP_COUNT = 4
IMPORT_COUNT = 100000 + PREVIOUS_TIMESTEP_COUNT

## Generate deterministic dataset

In [20]:
import hmac
import hashlib

def hashdice(secret_key, message):
    hmac_sha512 = hmac.new(key=secret_key.encode(), msg=message.encode(), digestmod=hashlib.sha512).hexdigest()
    lucky = int(hmac_sha512[0:5], 16)
    return lucky % 100000


with open(HD_OUTPUT_FILENAME, 'w') as f:

    for i in range(IMPORT_COUNT + 1):
        secret_key = '4671f003b18db91be8f1570c7f681ad9e97026c5f34d89a733f48ab81eb8d592'
        message = f'5533idsczGPRYacG5N15jfJ6nm5:{i}'
        result = hashdice(secret_key, message)
        rs = str(result)
        f.write(f'{rs + str(i%10000)}\n')

## Convert to input and target data

In [21]:
import numpy as np

# convert the sequence of generated numbers to 4 inputs and one output
def strided(a, L):
    shp = a.shape
    s  = a.strides
    nd0 = shp[0]-L+1
    shp_in = (nd0,L)+shp[1:]
    strd_in = (s[0],) + s
    return np.lib.stride_tricks.as_strided(a, shape=shp_in, strides=strd_in)

df = np.genfromtxt(HD_OUTPUT_FILENAME,delimiter='\n',dtype='uint64')
TOTAL_DATA_NUM = IMPORT_COUNT-PREVIOUS_TIMESTEP_COUNT

# calculates how many bits are in the output.
BIT_WIDTH = np.ceil(np.log2(np.amax(df))).astype(int)
print(BIT_WIDTH)
# convert the generated numbers to binary sequences
df_as_bits =(df[:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int)
df_as_frames = strided(df_as_bits, PREVIOUS_TIMESTEP_COUNT+1)

#indicies = np.arange(TOTAL_DATA_NUM,dtype='uint64')
#print(indicies)
#np.random.shuffle(indicies)
#print(indicies)
#df_as_frames=df_as_frames[indicies]

# convert the data into inputs and outputs
y = df_as_frames[:,-1,:]
X = df_as_frames[:,:-1,]
X = X.reshape([X.shape[0], X.shape[1]*X.shape[2]])

print(np.shape(X), np.shape(y))


print(X[0])
print(y[0])

30
(100001, 120) (100001, 30)
[0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0
 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]
[0 1 0 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0]


## Load pre-trained model

In [22]:
from tensorflow.keras.models import load_model
model = load_model('hashdice_model.h5')

## Validate bitwise accuracy

In [23]:
results = model.evaluate(X, y, batch_size=256)
print("test loss: %f, test acc: %s" % tuple(results))

391/391 [==============================] - 1s 3ms/step - loss: 0.5921 - binary_accuracy: 0.5989
test loss: 0.592145, test acc: 0.5989378094673157


In [35]:
import tensorflow as tf
index = 10
inputTest=np.reshape(X[index],(1,120))
predict = model.predict(inputTest)

1/1 [==============================] - 0s 132ms/step


In [36]:
orignalResult = ', '.join([str(x) for x in predict[0]])
print(f'Original Result {orignalResult}')
orignalResult = ', '.join([str(x) for x in predict[0][::-1]])
print(f'-------------------------')
print(f'Original Result {orignalResult}')
npWhere=np.where(predict[0] > 0.2, 1, 0)
print(f'reverse correct output------------{y[index]}')
print(f'predicted output reverse----------{npWhere}')
o = y[index][::-1]
print(f'correct output--------------------{o}')
npWhere = npWhere[::-1]
print(f'predicted output------------------{npWhere}')
tfround = tf.round(predict[0])
#print(f'tfround {tfround}')

output = sum(bit * 2**(len(o)-i-1) for i, bit in enumerate(o))
print(f'correct output--------------------{output}')
result = np.array(tfround[::-1])
result = sum(bit * 2**(len(result)-i-1) for i, bit in enumerate(result))
print(f'predicted output------------------{result}')

Original Result 2.370437e-07, 0.9999604, 0.6775574, 0.55164695, 0.5111164, 0.48322776, 0.5315596, 0.5957751, 0.57050997, 0.4798604, 0.44515052, 0.4681123, 0.5211956, 0.5018133, 0.51835114, 0.5521113, 0.5989501, 0.50600857, 0.51195425, 0.49408752, 0.44161886, 0.3734192, 0.40087196, 0.122052856, 0.0021829158, 0.0015598203, 0.0012291981, 1.2325062e-05, 1.9887497e-05, 1.8105751e-05
-------------------------
Original Result 1.8105751e-05, 1.9887497e-05, 1.2325062e-05, 0.0012291981, 0.0015598203, 0.0021829158, 0.122052856, 0.40087196, 0.3734192, 0.44161886, 0.49408752, 0.51195425, 0.50600857, 0.5989501, 0.5521113, 0.51835114, 0.5018133, 0.5211956, 0.4681123, 0.44515052, 0.4798604, 0.57050997, 0.5957751, 0.5315596, 0.48322776, 0.5111164, 0.55164695, 0.6775574, 0.9999604, 2.370437e-07
reverse correct output------------[0 1 1 0 0 0 1 1 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0]
predicted output reverse----------[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
correct output------

In [54]:
#000000000011010100001101001010
#000000000011011110100001111110